In [45]:
import random
import networkx as nx
import matplotlib.pyplot as plt

In [46]:
MIN_PER_RANK = 1  # Nodes/Rank: How 'fat' the DAG should be.
MAX_PER_RANK = 5
MIN_RANKS = 3     # Ranks: How 'tall' the DAG should be.
MAX_RANKS = 5
PERCENT = 30      # Chance of having an Edge.

def generate_dag(min_w, max_w, total_nodes): # min_w, max_w: node values range, total_nodes: total number of nodes
    random.seed()  # Initialize the random number generator

    G = nx.DiGraph()

    current_nodes = 0 # Total number of nodes in the graph
    ranks = [] # Number of nodes in each rank

    # Generate ranks with nodes until the total number of nodes is reached
    while current_nodes < total_nodes:
        new_nodes = min(MAX_PER_RANK, total_nodes - current_nodes) # Number of nodes in the new rank
        ranks.append(new_nodes) # Add the new rank to the list of ranks
        current_nodes += new_nodes # Update the total number of nodes

    nodes = 1 # Total number of nodes in the graph starts from 1

    for rank in ranks:
        for k in range(rank):
            # Assign a random weight to each new node
            node_weight = random.randint(min_w, max_w)
            G.add_node(nodes + k, weight=node_weight)

        # Edges from old nodes ('nodes') to new ones ('rank').
        for j in range(nodes - 1): # Adjusted to start from 0
            for k in range(rank):
                if random.randint(0, 99) < PERCENT: # Randomly decide if there is an edge between the nodes
                    G.add_edge(j + 1, k + nodes) # Adjusted to start from 1

        nodes += rank  # Accumulate into old node set.

    # remove isolated nodes
    G.remove_nodes_from(list(nx.isolates(G)))

    root_id = 0 # Root node is 0

    roots = [node for node in G.nodes() if G.in_degree(node) == 0] # Find the root nodes
    for _ in roots:
        G.add_edge(root_id, _) # Add an edge from the root node to each root node

    # add an attribute 'out' to the root node as a set containing 0
    G.nodes[root_id]['out'] = set()
    G.nodes[root_id]['out'].add(0)
    G.nodes[root_id]['weight'] = 0

    return G

In [47]:
# def compute_node_out(G, node_id):
#     node_weight = G.nodes[node_id]['weight']
#     predecessors = list(G.predecessors(node_id))

#     if 'out' in G.nodes[node_id]: # Check if the node already has an 'out' attribute
#         existing_out = G.nodes[node_id]['out']
#         if existing_out and predecessors:  # Check if the 'out' attribute is not empty and the node is not the root
#             raise ValueError(f"Node {node_id} already has a non-empty 'out': {existing_out} and it's not the root")
#         elif existing_out:  # Root node optimization
#             return existing_out  # No need to recompute

#     node_out = set()
#     for predecessor in predecessors:
#         node_out.update(x + node_weight for x in G.nodes[predecessor]['out'])

#     return node_out

def compute_node_out(G, node_id):
    node_weight = G.nodes[node_id]['weight']
    predecessors = list(G.predecessors(node_id))

    node_out = set()
    for predecessor in predecessors:
        predecessor_out = G.nodes[predecessor]['out']
        node_out = node_out.union(predecessor_out)

    node_out = set([x + node_weight for x in node_out])
    return node_out

def process_graph(G):
    for node in list(nx.topological_sort(G))[1::]: # the root is already initialized with (0)
        G.nodes[node]['out'] = compute_node_out(G, node)

def rank(G, node_id):
    rank =[]
    for element in G.nodes[node_id]['out']:
        rank.append((element - G.nodes[node_id]['weight'] + 1, element))

    rank = sorted(rank, key=lambda x: x[0])

    return rank

In [49]:
# random generated DAG
G = generate_dag(1, 10, 20) #(min weight, max weight, total number of nodes)
process_graph(G)

for node in G.nodes():
    print(f"Node {node} weight: {G.nodes[node]['weight']} \tout: {sorted(list(G.nodes[node]['out']))}")

Node 1 weight: 2 	out: [2]
Node 2 weight: 4 	out: [4]
Node 3 weight: 2 	out: [2]
Node 4 weight: 3 	out: [3]
Node 5 weight: 6 	out: [6]
Node 6 weight: 7 	out: [9, 10, 13]
Node 7 weight: 9 	out: [9]
Node 8 weight: 7 	out: [9]
Node 9 weight: 3 	out: [6]
Node 10 weight: 10 	out: [12, 16]
Node 11 weight: 6 	out: [8, 10, 12, 15]
Node 12 weight: 8 	out: [12, 14, 17, 18, 21]
Node 13 weight: 3 	out: [5, 9, 12, 13, 16]
Node 14 weight: 7 	out: [13, 19, 23]
Node 15 weight: 3 	out: [7, 9]
Node 16 weight: 10 	out: [16, 19, 20, 22, 23, 26]
Node 17 weight: 7 	out: [10, 12, 13, 16, 17, 19, 20, 23, 26, 30]
Node 18 weight: 10 	out: [13, 16, 19]
Node 19 weight: 5 	out: [11, 13, 15, 17, 18, 20, 21, 24, 28]
Node 20 weight: 3 	out: [11, 12, 13, 15, 17, 18, 20, 21, 24]
Node 0 weight: 0 	out: [0]


In [52]:
rank_6 = rank(G, 6)
rank_10 = rank(G, 10)

print(f"Rank of node 6: {rank_6}")
print(f"Rank of node 10: {rank_10}")

Rank of node 6: [(3, 9), (4, 10), (7, 13)]
Rank of node 10: [(3, 12), (7, 16)]
